# dType Data Balancing on Shards

In [102]:
import random

In [103]:
shard_count = 20
dtype_count = 100
max_shard_load = 400
max_dtype_load = 2000
average_coef = 1.2

In [104]:
shard_loads_initial = list(enumerate([random.randrange(i, max_shard_load) for i in range(shard_count)]))
dtype_loads_initial = list(enumerate([random.randrange(i, max_dtype_load) for i in range(dtype_count)]))
shards = [[] for i in range(shard_count)]

print('shard_loads_initial', shard_loads_initial)
print('dtype_loads_initial', dtype_loads_initial)

shard_loads_initial [(0, 218), (1, 174), (2, 210), (3, 275), (4, 224), (5, 129), (6, 59), (7, 382), (8, 260), (9, 306), (10, 231), (11, 82), (12, 72), (13, 161), (14, 152), (15, 308), (16, 334), (17, 317), (18, 101), (19, 29)]
dtype_loads_initial [(0, 1100), (1, 1467), (2, 701), (3, 1917), (4, 1028), (5, 741), (6, 1356), (7, 1849), (8, 1158), (9, 709), (10, 560), (11, 1751), (12, 1682), (13, 1850), (14, 617), (15, 1922), (16, 1516), (17, 488), (18, 458), (19, 167), (20, 1761), (21, 280), (22, 1664), (23, 845), (24, 639), (25, 64), (26, 880), (27, 603), (28, 1740), (29, 1613), (30, 950), (31, 1563), (32, 1198), (33, 669), (34, 1391), (35, 427), (36, 1372), (37, 87), (38, 1919), (39, 1726), (40, 1171), (41, 46), (42, 1342), (43, 226), (44, 981), (45, 813), (46, 536), (47, 1819), (48, 1062), (49, 1955), (50, 160), (51, 494), (52, 1023), (53, 460), (54, 1206), (55, 1257), (56, 1150), (57, 719), (58, 253), (59, 319), (60, 1261), (61, 1034), (62, 1875), (63, 100), (64, 1746), (65, 228), (66,

In [105]:
# Calculate average count per shard
average_load_shard = (sum(i[1] for i in dtype_loads) + sum(i[1] for i in shard_loads)) / shard_count
average_load_shard *= average_coef

print('average_load_shard', average_load_shard)

average_load_shard 8891.82


In [106]:
next_index_s = 0
next_index_dt = 0
last_index_dt = len(dtype_loads_initial) - 1
last_index_s = len(shard_loads_initial) - 1

# Sort first
shard_loads = sorted(shard_loads_initial, key=lambda tup: tup[1])
dtype_loads = sorted(dtype_loads_initial, key=lambda tup: tup[1], reverse=True)

# Move heavier than average dtypes on the least heaviest shards
for i, dload in dtype_loads:
    if dload >= average_load_shard:
        shards[next_index_s].append(i)
        next_index_s += 1
        next_index_dt += 1


# 
for i, dload in dtype_loads[next_index_dt:]:
    if last_index_s < next_index_s:
        print('Needs more shards. Increase average_coef');
        break
    shards[next_index_s].append(i)
    load = shard_loads[next_index_s][1] + dload + dtype_loads[last_index_dt][1]
    while last_index_dt > next_index_dt and load <= average_load_shard:
        shards[next_index_s].append(dtype_loads[last_index_dt][0])
        last_index_dt -= 1
        load += dtype_loads[last_index_dt][1]
    next_index_s += 1
    next_index_dt += 1
    if next_index_dt > last_index_dt:
        break

print('(shard_index, shard_load, dtype_indexes)')
final_shards = [(shard_loads[x][0], sum([dtype_loads_initial[dtype_index][1] for dtype_index in shards[x]]), shards[x]) for x, _ in enumerate(shards)]
print('final_shards', final_shards)

(shard_index, shard_load, dtype_indexes)
final_shards [(19, 8639, [49, 41, 25, 74, 37, 87, 63, 84, 67, 50, 19, 43, 65, 71, 68, 93, 58, 96, 21, 59, 92, 77, 69, 98, 35, 90, 18, 53]), (6, 8151, [15, 88, 17, 51, 66, 46, 10, 27, 14, 75, 24, 33]), (12, 8632, [38, 2, 9, 57, 99, 5, 81, 85, 86, 45]), (11, 8388, [3, 23, 97, 26, 73, 30, 44, 76]), (18, 8272, [62, 52, 4, 61, 48, 0, 56]), (5, 7717, [13, 80, 8, 40, 95, 32]), (14, 8144, [7, 54, 55, 60, 82, 79]), (13, 8548, [72, 94, 91, 42, 6, 36]), (1, 7649, [47, 34, 89, 1, 70]), (2, 7966, [20, 83, 16, 31, 29]), (0, 8534, [11, 22, 12, 78, 39]), (4, 3486, [64, 28]), (10, 0, []), (8, 0, []), (3, 0, []), (9, 0, []), (15, 0, []), (17, 0, []), (16, 0, []), (7, 0, [])]
